# seed 0 Warm up + Batch norm gamma setting (layer 4 bottleneck index 2)

In [1]:
# Author: Sasank Chilamkurthy

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import numpy as np
from openpyxl import Workbook
from warmup_scheduler import GradualWarmupScheduler
# from pytorch_cosine_annealing_warmup_master import cosine_annearing_with_warmup
import random


plt.ion()   # interactive mode

#torch.set_deterministic(True)
random_seed = 0
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.ColorJitter(brightness=(0.8, 1.2), contrast=(0.8, 1.3)),
        transforms.RandomAffine(degrees=0, translate=(0.15,0.1)),
        transforms.RandomAffine(degrees=(-10,10)),
        transforms.Resize(280),
        transforms.CenterCrop(224),
        #transforms.Resize(size=(224,224)),
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        transforms.Normalize([0.493, 0.493, 0.493], [0.246, 0.246, 0.246])
    ]),
    'val': transforms.Compose([
        #transforms.Resize(size=(224,224)),
        transforms.Resize(280),
        transforms.CenterCrop(224),
        #transforms.Resize(size=(224,224)),
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        transforms.Normalize([0.493, 0.493, 0.493], [0.246, 0.246, 0.246])
    ]),
}

data_dir = '/home/aiffel/Hackathon_covid_19/data_2'
#data_dir = 'Crop_CXR/binary_covid'
batch_size = 16
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                             shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
val_class_names = image_datasets['val'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print("Training dataset size: " + str(dataset_sizes['train']))
print("Validation dataset size: " +str(dataset_sizes['val']))
print(val_class_names)


val_dir = '/home/aiffel/Hackathon_covid_19/data_2/val/'
false_data_dir = val_dir + 'false'
pneumonia_data_dir = val_dir + 'pneumonia'

print("######### Validation Dataset #########")
val_false_num = len(os.listdir(false_data_dir))
val_pneumonia_num = len(os.listdir(pneumonia_data_dir))
print("false size: " + str(val_false_num))
print("pneumonia size: " + str(val_pneumonia_num))

wb = Workbook()      # 워크북을 생성한다.
ws = wb.active       # 워크 시트를 얻는다.
    
ws['A1'] = 'ResNet50'
ws['B1'] = 'Val ACC'

ws['D1'] = 'False ACC'
ws['E1'] = 'False Recall'
ws['F1'] = 'False Precision'
ws['G1'] = 'False F1'

ws['I1'] = 'Penumonia ACC'
ws['J1'] = 'Penumonia Recall'
ws['K1'] = 'Penumonia Precision'
ws['L1'] = 'Penumonia F1'


def train_model(model, criterion, optimizer, scheduler, num_epochs):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    best_false_acc = 0.0
    best_false_f1 = 0.0
    best_pneumonia_acc = 0.0
    best_pneumonia_f1 = 0.0

    best_false_acc_epoch = 0
    best_false_f1_epoch = 0
    best_pneumonia_acc_epoch = 0
    best_pneumonia_f1_epoch = 0

    for epoch in range(num_epochs):
        val_false_TP = 0.0
        val_pneumonia_TP = 0.0
        val_false_FN = 0.0
        val_pneumonia_FN = 0.0
        val_false_TN = 0.0
        val_pneumonia_TN = 0.0
        val_false_FP = 0.0
        val_pneumonia_FP = 0.0

        
        
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        A = 'A'+str(epoch+2)
        ws[A] = 'Epoch' + str(epoch+1)
    
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    #outputs = torch.nn.functional.softmax(outputs, dim=1)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    # Append batch prediction results

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
                a = preds.size()
                b = a[0]
                
                if phase == 'val':
                    for i in range(b):
                        if preds[i].item() == labels.data[i].item():
                            #Normal-index:0, Pneumonia-index:1
                            if preds.data[i].item() == 0: 
                                val_false_TP += 1 #Normal 관점에서는 normal를 정확히 분류하는 것이 TP
                                val_pneumonia_TN += 1 #Pneumonia 관점에서는 normal를 정확히 분류하는 것이 TN
                            elif preds.data[i].item() == 1: 
                                val_pneumonia_TP += 1 #Pneumonia 관점에서는 Pneumonia를 정확히 분류하는 것이 TP
                                val_false_TN += 1 #Normal 관점에서는 Pneumonia를 정확히 분류하는 것이 TN

                        if preds[i].item() != labels.data[i].item():
                            if preds.data[i].item() == 0:
                                val_false_FP += 1 #Normal 관점에서 normal라고 분류했지만 실제로는 pneumonia인 경우는 FP
                                val_pneumonia_FN += 1 #Pneumonia 관점에서 normal라고 분류했지만 실제로는 pneumonia인 경우는 FN
                            elif preds.data[i].item() == 1:
                                val_pneumonia_FP += 1 #Normal 관점에서 pneumonia라고 분류했지만 실제로는 normal인 경우는 FP
                                val_false_FN += 1 #Pneumonia 관점에서 pneumonia라고 분류했지만 실제로는 normal인 경우는 FN

            if phase == 'train':
                scheduler.step()
                
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
           
            if phase == 'val':
                B = 'B'+str(epoch+2)
                ws[B] = '{:.4f}'.format(epoch_acc*100)

                print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, (epoch_acc*100)))    

            
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            

        #recall, precision -> https://en.wikipedia.org/wiki/Precision_and_recall
        #recall = TP/(TP+FN)
        #precision = TP/(TP+FP)
        #ACC = (TP+TN)/(TP+TN+FP+FN)
        false_recall = round(val_false_TP/(val_false_TP + val_false_FN),4)*100
        pneumonia_recall = round(val_pneumonia_TP/(val_pneumonia_TP + val_pneumonia_FN),4)*100
        false_precision = round(val_false_TP/(val_false_TP + val_false_FP),4)*100
        pneumonia_precision = round(val_pneumonia_TP/(val_pneumonia_TP + val_pneumonia_FP),4)*100
        false_acc = round((val_false_TP + val_false_TN)/(val_false_TP + val_false_FP +val_false_TN + val_false_FN),4)*100
        pneumonia_acc = round((val_pneumonia_TP + val_pneumonia_TN)/(val_pneumonia_TP + val_pneumonia_FP +val_pneumonia_TN + val_pneumonia_FN),4)*100
        #F1 score -> https://en.wikipedia.org/wiki/F-score
        #F1 score = 2/((1/recall)+(1/precision)) = 2((precision*recall)/(precision+recall)) = tp/(tp+((1/2)(fp+fn)))
        false_f1_score = round(2*(false_precision*false_recall)/(false_precision+false_recall),2)
        pneumonia_f1_score = round(2*(pneumonia_precision*pneumonia_recall)/(pneumonia_precision+pneumonia_recall),2)

        #total_data = batch size --> covid_total_dataset = false_total_dataset
        false_total_dataset = int(val_false_TP + val_false_FP +val_false_TN + val_false_FN)
        pneumonia_total_dataset = int(val_pneumonia_TP + val_pneumonia_FP +val_pneumonia_TN + val_pneumonia_FN)
        
      
        D = 'D'+str(epoch+2)
        E = 'E'+str(epoch+2)
        F = 'F'+str(epoch+2)
        G = 'G'+str(epoch+2)
        
        I = 'I'+str(epoch+2)
        J = 'J'+str(epoch+2)
        K = 'K'+str(epoch+2)
        L = 'L'+str(epoch+2)

        ws[D] = false_acc
        ws[E] = false_recall
        ws[F] = false_precision
        ws[G] = false_f1_score
        
        ws[I] = pneumonia_acc
        ws[J] = pneumonia_recall
        ws[K] = pneumonia_precision
        ws[L] = pneumonia_f1_score


        print()

        print('데이터셋A: ' + str(false_total_dataset))
        print('False Recall: ' + str(false_recall))        
        print('False Precision: ' + str(false_precision))
        print('False ACC: ' + str(false_acc))
        print('False F1 score: ' + str(false_f1_score))
        
        print()
        
        print('데이터셋B: ' + str(pneumonia_total_dataset))
        print('Pneumonia Recall: ' + str(pneumonia_recall))
        print('Pneumonia Precision: ' + str(pneumonia_precision))
        print('Pneumonia ACC: ' + str(pneumonia_acc))
        print('Pneumonia F1 score: ' + str(pneumonia_f1_score))
        
  
        if phase == 'val' and false_acc > best_false_acc:
            best_false_acc = false_acc
            best_false_acc_epoch = epoch

        if phase == 'val' and pneumonia_acc > best_pneumonia_acc:
            best_pneumonia_acc = pneumonia_acc
            best_pneumonia_acc_epoch = epoch
            

        if phase == 'val' and false_f1_score > best_false_f1:
            best_false_f1 = false_f1_score
            best_false_f1_epoch = epoch
            
        if phase == 'val' and pneumonia_f1_score > best_pneumonia_f1:
            best_pneumonia_f1 = pneumonia_f1_score
            best_pneumonia_f1_epoch = epoch

        print()
        print()

    
    ws['N1'] = 'Best False ACC'
    ws['O1'] = 'epoch'
    ws['N2'] = round(best_false_acc,2)
    ws['O2'] = best_false_acc_epoch+1
        
    ws['N4'] = 'Best False F1'
    ws['O4'] = 'epoch'
    ws['N5'] = round(best_false_f1,2)
    ws['O5'] = best_false_f1_epoch+1
        
    ws['N7'] = 'Best Pneumonia ACC'
    ws['O7'] = 'epoch'
    ws['N8'] = round(best_pneumonia_acc,2)
    ws['O8'] = best_pneumonia_acc_epoch+1
        
    ws['N10'] = 'Best Pneumonia F1'
    ws['O10'] = 'epoch'
    ws['N11'] = round(best_pneumonia_f1,2)
    ws['O11'] = best_pneumonia_f1_epoch+1

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    #print('Best val Acc: {:4f}'.format(best_acc))


   

    # load best model weights
    #model.load_state_dict(best_covid_wts)
    wb.save('output_excel/현수님세팅_Pneumonia_vs_Fasle(train5475_P 기준세팅)_Balanced(val)_seed_0_BatchNorm_layer_4_bottleneck_2.xlsx') # 엑셀로 저장한다. 
    #torch.save(model.state_dict(), 'covid_binary.pt')
    return model


model_ft = models.resnet50(pretrained=True)

###BatchNorm Gamma setting###


# seed 0,1,2

# count =0
# bottleneck_count = 0
# layer_index=1
# layer_name = 'layer'+str(layer_index)
# bottleneck_index = 0



# count =0
# bottleneck_count = 0
# layer_index=4
# layer_name = 'layer'+str(layer_index)
# bottleneck_index = 0


count =0
bottleneck_count = 0
layer_index=4
layer_name = 'layer'+str(layer_index)
bottleneck_index = 2



for name, layer in model_ft.named_children():
    if bottleneck_count > 0:
        bottleneck_index = 0
    if name == layer_name:
        bn_index = 1
        for name, param in model_ft.named_parameters():
            if name == layer_name + '.' + str(bottleneck_index) + '.' + 'bn1.weight':
                print(str(layer_name + "의 " + str(bottleneck_index)+ "번째-bottleneck의 " + 'bn1.weight'))
                torch.nn.init.ones_(param)
                print(name + '의 gammm one setting 완료')
                print()
                
            elif name == layer_name + '.' + str(bottleneck_index) + '.' + 'bn2.weight':
                print(str(layer_name + "의 " + str(bottleneck_index)+ "번째-bottleneck의 " + 'bn2.weight'))
                torch.nn.init.ones_(param)
                print(name + '의 gammm one setting 완료')
                print()

            elif name == layer_name + '.' + str(bottleneck_index) + '.' + 'bn3.weight':
                print(str(layer_name + "의 " + str(bottleneck_index)+ "번째-bottleneck의 " + 'bn3.weight'))
                torch.nn.init.zeros_(param)
                print(name + '의 gammm zero setting 완료')
                print()

            elif name == layer_name + '.' + str(bottleneck_index) + '.' + 'bn' + str(bn_index) + '.bias':
                print(str(layer_name + "의 " + str(bottleneck_index)+ "번째-bottleneck의 " + 'bn' + str(bn_index) + '.bias'))
                torch.nn.init.zeros_(param)
                bn_index = bn_index + 1
                if name == layer_name + '.' + str(bottleneck_index) + '.' + 'bn3.bias':
                    bottleneck_index = bottleneck_index + 1
                    bn_index = 1
                print(name + '의 beta zero setting 완료')
                print()

        bottleneck_count = bottleneck_count + 1
        layer_index = layer_index+ 1
        layer_name = 'layer'+str(layer_index)     

for param in model_ft.parameters():
    param.requires_grad = True

num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 2)




model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=3, gamma=0.1)
#exp_lr_scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max, eta_min=0, last_epoch=-1, verbose=False)
exp_lr_scheduler = GradualWarmupScheduler(optimizer_ft, multiplier=1, total_epoch=5, after_scheduler=exp_lr_scheduler)
#exp_lr_scheduler = cosine_annearing_with_warmup.CosineAnnealingWarmUpRestarts(optimizer_ft, T_0=30, T_mult=1, eta_max=0.001, T_up=20)

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=20)

# freeze backbone layers
'''
count = 0
for child_name, param in model_ft.named_children():
    for param_name, param_param in model_ft.named_parameters():
        if child_name == 'layer4': 
            param_param.requires_grad = True
        else:
            param_param.requires_grad = False
'''


#특정 conv layer 이후 초기화
'''
children_index = 4
children_name = 'layer'+str(children_index)
bottleneck_index = 2
for name, layer in model_ft.named_children():
    if name == children_name:
        conv_index = 1
        for name, param in model_ft.named_parameters():
            if name == children_name + "." + str(bottleneck_index) + '.' + 'conv' + str(conv_index) + '.weight':
                print(children_name + "의 "+ str(bottleneck_index)+ "번째-bottleneck의 " + 'conv' +str(conv_index) + '.weight')
                torch.nn.init.xavier_uniform_(param)
                print(name + '의 conv filter initilization setting 완료')
                print()
                conv_index = conv_index + 1
                if name == 'layer3.' + str(bottleneck_index) + '.' + 'conv3.weight':
                    bottleneck_index = bottleneck_index + 1
                    conv_index = 1
                elif name == 'layer4.' + str(bottleneck_index) + '.' + 'conv3.weight':
                    bottleneck_index = bottleneck_index + 1
                    conv_index = 1
        children_index = children_index + 1
        children_name = 'layer'+str(children_index)
        bottleneck_index = 0
'''        




#increase fine-tune layer per epoch
'''
if epoch < 6:
    # freeze backbone layers
    for param in net.parameters():
        count +=1
        if count < 4: #freezing first 3 layers
            param.requires_grad = False    
        else:
            for param in net.parameters():
                aram.requires_grad = True
'''

Training dataset size: 10950
Validation dataset size: 300
['false', 'pneumonia']
######### Validation Dataset #########
false size: 200
pneumonia size: 100
layer4의 2번째-bottleneck의 bn1.weight
layer4.2.bn1.weight의 gammm one setting 완료

layer4의 2번째-bottleneck의 bn1.bias
layer4.2.bn1.bias의 beta zero setting 완료

layer4의 2번째-bottleneck의 bn2.weight
layer4.2.bn2.weight의 gammm one setting 완료

layer4의 2번째-bottleneck의 bn2.bias
layer4.2.bn2.bias의 beta zero setting 완료

layer4의 2번째-bottleneck의 bn3.weight
layer4.2.bn3.weight의 gammm zero setting 완료

layer4의 2번째-bottleneck의 bn3.bias
layer4.2.bn3.bias의 beta zero setting 완료

Epoch 0/19
----------
val Loss: 0.6586 Acc: 67.0000

데이터셋A: 300
False Recall: 99.5
False Precision: 67.0
False ACC: 67.0
False F1 score: 80.08

데이터셋B: 300
Pneumonia Recall: 2.0
Pneumonia Precision: 66.67
Pneumonia ACC: 67.0
Pneumonia F1 score: 3.88


Epoch 1/19
----------
val Loss: 0.4682 Acc: 75.3333

데이터셋A: 300
False Recall: 67.0
False Precision: 94.37
False ACC: 75.33
False F1 scor

'\nif epoch < 6:\n    # freeze backbone layers\n    for param in net.parameters():\n        count +=1\n        if count < 4: #freezing first 3 layers\n            param.requires_grad = False    \n        else:\n            for param in net.parameters():\n                aram.requires_grad = True\n'

# seed 1 Warm up + Batch norm gamma setting (layer 4 bottleneck index 2)

In [2]:
# Author: Sasank Chilamkurthy

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import numpy as np
from openpyxl import Workbook
from warmup_scheduler import GradualWarmupScheduler
# from pytorch_cosine_annealing_warmup_master import cosine_annearing_with_warmup
import random


plt.ion()   # interactive mode

#torch.set_deterministic(True)
random_seed = 1
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.ColorJitter(brightness=(0.8, 1.2), contrast=(0.8, 1.3)),
        transforms.RandomAffine(degrees=0, translate=(0.15,0.1)),
        transforms.RandomAffine(degrees=(-10,10)),
        transforms.Resize(280),
        transforms.CenterCrop(224),
        #transforms.Resize(size=(224,224)),
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        transforms.Normalize([0.493, 0.493, 0.493], [0.246, 0.246, 0.246])
    ]),
    'val': transforms.Compose([
        #transforms.Resize(size=(224,224)),
        transforms.Resize(280),
        transforms.CenterCrop(224),
        #transforms.Resize(size=(224,224)),
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        transforms.Normalize([0.493, 0.493, 0.493], [0.246, 0.246, 0.246])
    ]),
}

data_dir = '/home/aiffel/Hackathon_covid_19/data_2'
#data_dir = 'Crop_CXR/binary_covid'
batch_size = 16
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                             shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
val_class_names = image_datasets['val'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print("Training dataset size: " + str(dataset_sizes['train']))
print("Validation dataset size: " +str(dataset_sizes['val']))
print(val_class_names)


val_dir = '/home/aiffel/Hackathon_covid_19/data_2/val/'
false_data_dir = val_dir + 'false'
pneumonia_data_dir = val_dir + 'pneumonia'

print("######### Validation Dataset #########")
val_false_num = len(os.listdir(false_data_dir))
val_pneumonia_num = len(os.listdir(pneumonia_data_dir))
print("false size: " + str(val_false_num))
print("pneumonia size: " + str(val_pneumonia_num))

wb = Workbook()      # 워크북을 생성한다.
ws = wb.active       # 워크 시트를 얻는다.
    
ws['A1'] = 'ResNet50'
ws['B1'] = 'Val ACC'

ws['D1'] = 'False ACC'
ws['E1'] = 'False Recall'
ws['F1'] = 'False Precision'
ws['G1'] = 'False F1'

ws['I1'] = 'Penumonia ACC'
ws['J1'] = 'Penumonia Recall'
ws['K1'] = 'Penumonia Precision'
ws['L1'] = 'Penumonia F1'


def train_model(model, criterion, optimizer, scheduler, num_epochs):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    best_false_acc = 0.0
    best_false_f1 = 0.0
    best_pneumonia_acc = 0.0
    best_pneumonia_f1 = 0.0

    best_false_acc_epoch = 0
    best_false_f1_epoch = 0
    best_pneumonia_acc_epoch = 0
    best_pneumonia_f1_epoch = 0

    for epoch in range(num_epochs):
        val_false_TP = 0.0
        val_pneumonia_TP = 0.0
        val_false_FN = 0.0
        val_pneumonia_FN = 0.0
        val_false_TN = 0.0
        val_pneumonia_TN = 0.0
        val_false_FP = 0.0
        val_pneumonia_FP = 0.0

        
        
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        A = 'A'+str(epoch+2)
        ws[A] = 'Epoch' + str(epoch+1)
    
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    #outputs = torch.nn.functional.softmax(outputs, dim=1)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    # Append batch prediction results

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
                a = preds.size()
                b = a[0]
                
                if phase == 'val':
                    for i in range(b):
                        if preds[i].item() == labels.data[i].item():
                            #Normal-index:0, Pneumonia-index:1
                            if preds.data[i].item() == 0: 
                                val_false_TP += 1 #Normal 관점에서는 normal를 정확히 분류하는 것이 TP
                                val_pneumonia_TN += 1 #Pneumonia 관점에서는 normal를 정확히 분류하는 것이 TN
                            elif preds.data[i].item() == 1: 
                                val_pneumonia_TP += 1 #Pneumonia 관점에서는 Pneumonia를 정확히 분류하는 것이 TP
                                val_false_TN += 1 #Normal 관점에서는 Pneumonia를 정확히 분류하는 것이 TN

                        if preds[i].item() != labels.data[i].item():
                            if preds.data[i].item() == 0:
                                val_false_FP += 1 #Normal 관점에서 normal라고 분류했지만 실제로는 pneumonia인 경우는 FP
                                val_pneumonia_FN += 1 #Pneumonia 관점에서 normal라고 분류했지만 실제로는 pneumonia인 경우는 FN
                            elif preds.data[i].item() == 1:
                                val_pneumonia_FP += 1 #Normal 관점에서 pneumonia라고 분류했지만 실제로는 normal인 경우는 FP
                                val_false_FN += 1 #Pneumonia 관점에서 pneumonia라고 분류했지만 실제로는 normal인 경우는 FN

            if phase == 'train':
                scheduler.step()
                
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
           
            if phase == 'val':
                B = 'B'+str(epoch+2)
                ws[B] = '{:.4f}'.format(epoch_acc*100)

                print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, (epoch_acc*100)))    

            
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            

        #recall, precision -> https://en.wikipedia.org/wiki/Precision_and_recall
        #recall = TP/(TP+FN)
        #precision = TP/(TP+FP)
        #ACC = (TP+TN)/(TP+TN+FP+FN)
        false_recall = round(val_false_TP/(val_false_TP + val_false_FN),4)*100
        pneumonia_recall = round(val_pneumonia_TP/(val_pneumonia_TP + val_pneumonia_FN),4)*100
        false_precision = round(val_false_TP/(val_false_TP + val_false_FP),4)*100
        pneumonia_precision = round(val_pneumonia_TP/(val_pneumonia_TP + val_pneumonia_FP),4)*100
        false_acc = round((val_false_TP + val_false_TN)/(val_false_TP + val_false_FP +val_false_TN + val_false_FN),4)*100
        pneumonia_acc = round((val_pneumonia_TP + val_pneumonia_TN)/(val_pneumonia_TP + val_pneumonia_FP +val_pneumonia_TN + val_pneumonia_FN),4)*100
        #F1 score -> https://en.wikipedia.org/wiki/F-score
        #F1 score = 2/((1/recall)+(1/precision)) = 2((precision*recall)/(precision+recall)) = tp/(tp+((1/2)(fp+fn)))
        false_f1_score = round(2*(false_precision*false_recall)/(false_precision+false_recall),2)
        pneumonia_f1_score = round(2*(pneumonia_precision*pneumonia_recall)/(pneumonia_precision+pneumonia_recall),2)

        #total_data = batch size --> covid_total_dataset = false_total_dataset
        false_total_dataset = int(val_false_TP + val_false_FP +val_false_TN + val_false_FN)
        pneumonia_total_dataset = int(val_pneumonia_TP + val_pneumonia_FP +val_pneumonia_TN + val_pneumonia_FN)
        
      
        D = 'D'+str(epoch+2)
        E = 'E'+str(epoch+2)
        F = 'F'+str(epoch+2)
        G = 'G'+str(epoch+2)
        
        I = 'I'+str(epoch+2)
        J = 'J'+str(epoch+2)
        K = 'K'+str(epoch+2)
        L = 'L'+str(epoch+2)

        ws[D] = false_acc
        ws[E] = false_recall
        ws[F] = false_precision
        ws[G] = false_f1_score
        
        ws[I] = pneumonia_acc
        ws[J] = pneumonia_recall
        ws[K] = pneumonia_precision
        ws[L] = pneumonia_f1_score


        print()

        print('데이터셋A: ' + str(false_total_dataset))
        print('False Recall: ' + str(false_recall))        
        print('False Precision: ' + str(false_precision))
        print('False ACC: ' + str(false_acc))
        print('False F1 score: ' + str(false_f1_score))
        
        print()
        
        print('데이터셋B: ' + str(pneumonia_total_dataset))
        print('Pneumonia Recall: ' + str(pneumonia_recall))
        print('Pneumonia Precision: ' + str(pneumonia_precision))
        print('Pneumonia ACC: ' + str(pneumonia_acc))
        print('Pneumonia F1 score: ' + str(pneumonia_f1_score))
        
  
        if phase == 'val' and false_acc > best_false_acc:
            best_false_acc = false_acc
            best_false_acc_epoch = epoch

        if phase == 'val' and pneumonia_acc > best_pneumonia_acc:
            best_pneumonia_acc = pneumonia_acc
            best_pneumonia_acc_epoch = epoch
            

        if phase == 'val' and false_f1_score > best_false_f1:
            best_false_f1 = false_f1_score
            best_false_f1_epoch = epoch
            
        if phase == 'val' and pneumonia_f1_score > best_pneumonia_f1:
            best_pneumonia_f1 = pneumonia_f1_score
            best_pneumonia_f1_epoch = epoch

        print()
        print()

    
    ws['N1'] = 'Best False ACC'
    ws['O1'] = 'epoch'
    ws['N2'] = round(best_false_acc,2)
    ws['O2'] = best_false_acc_epoch+1
        
    ws['N4'] = 'Best False F1'
    ws['O4'] = 'epoch'
    ws['N5'] = round(best_false_f1,2)
    ws['O5'] = best_false_f1_epoch+1
        
    ws['N7'] = 'Best Pneumonia ACC'
    ws['O7'] = 'epoch'
    ws['N8'] = round(best_pneumonia_acc,2)
    ws['O8'] = best_pneumonia_acc_epoch+1
        
    ws['N10'] = 'Best Pneumonia F1'
    ws['O10'] = 'epoch'
    ws['N11'] = round(best_pneumonia_f1,2)
    ws['O11'] = best_pneumonia_f1_epoch+1

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    #print('Best val Acc: {:4f}'.format(best_acc))


   

    # load best model weights
    #model.load_state_dict(best_covid_wts)
    wb.save('output_excel/현수님세팅_Pneumonia_vs_Fasle(train5475_P 기준세팅)_Balanced(val)_seed_1_BatchNorm_layer_4_bottleneck_2.xlsx') # 엑셀로 저장한다. 
    #torch.save(model.state_dict(), 'covid_binary.pt')
    return model


model_ft = models.resnet50(pretrained=True)

###BatchNorm Gamma setting###


# seed 0,1,2

# count =0
# bottleneck_count = 0
# layer_index=1
# layer_name = 'layer'+str(layer_index)
# bottleneck_index = 0



# count =0
# bottleneck_count = 0
# layer_index=4
# layer_name = 'layer'+str(layer_index)
# bottleneck_index = 0


count =0
bottleneck_count = 0
layer_index=4
layer_name = 'layer'+str(layer_index)
bottleneck_index = 2



for name, layer in model_ft.named_children():
    if bottleneck_count > 0:
        bottleneck_index = 0
    if name == layer_name:
        bn_index = 1
        for name, param in model_ft.named_parameters():
            if name == layer_name + '.' + str(bottleneck_index) + '.' + 'bn1.weight':
                print(str(layer_name + "의 " + str(bottleneck_index)+ "번째-bottleneck의 " + 'bn1.weight'))
                torch.nn.init.ones_(param)
                print(name + '의 gammm one setting 완료')
                print()
                
            elif name == layer_name + '.' + str(bottleneck_index) + '.' + 'bn2.weight':
                print(str(layer_name + "의 " + str(bottleneck_index)+ "번째-bottleneck의 " + 'bn2.weight'))
                torch.nn.init.ones_(param)
                print(name + '의 gammm one setting 완료')
                print()

            elif name == layer_name + '.' + str(bottleneck_index) + '.' + 'bn3.weight':
                print(str(layer_name + "의 " + str(bottleneck_index)+ "번째-bottleneck의 " + 'bn3.weight'))
                torch.nn.init.zeros_(param)
                print(name + '의 gammm zero setting 완료')
                print()

            elif name == layer_name + '.' + str(bottleneck_index) + '.' + 'bn' + str(bn_index) + '.bias':
                print(str(layer_name + "의 " + str(bottleneck_index)+ "번째-bottleneck의 " + 'bn' + str(bn_index) + '.bias'))
                torch.nn.init.zeros_(param)
                bn_index = bn_index + 1
                if name == layer_name + '.' + str(bottleneck_index) + '.' + 'bn3.bias':
                    bottleneck_index = bottleneck_index + 1
                    bn_index = 1
                print(name + '의 beta zero setting 완료')
                print()

        bottleneck_count = bottleneck_count + 1
        layer_index = layer_index+ 1
        layer_name = 'layer'+str(layer_index)     

for param in model_ft.parameters():
    param.requires_grad = True

num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 2)




model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=3, gamma=0.1)
#exp_lr_scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max, eta_min=0, last_epoch=-1, verbose=False)
exp_lr_scheduler = GradualWarmupScheduler(optimizer_ft, multiplier=1, total_epoch=5, after_scheduler=exp_lr_scheduler)
#exp_lr_scheduler = cosine_annearing_with_warmup.CosineAnnealingWarmUpRestarts(optimizer_ft, T_0=30, T_mult=1, eta_max=0.001, T_up=20)

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=20)

# freeze backbone layers
'''
count = 0
for child_name, param in model_ft.named_children():
    for param_name, param_param in model_ft.named_parameters():
        if child_name == 'layer4': 
            param_param.requires_grad = True
        else:
            param_param.requires_grad = False
'''


#특정 conv layer 이후 초기화
'''
children_index = 4
children_name = 'layer'+str(children_index)
bottleneck_index = 2
for name, layer in model_ft.named_children():
    if name == children_name:
        conv_index = 1
        for name, param in model_ft.named_parameters():
            if name == children_name + "." + str(bottleneck_index) + '.' + 'conv' + str(conv_index) + '.weight':
                print(children_name + "의 "+ str(bottleneck_index)+ "번째-bottleneck의 " + 'conv' +str(conv_index) + '.weight')
                torch.nn.init.xavier_uniform_(param)
                print(name + '의 conv filter initilization setting 완료')
                print()
                conv_index = conv_index + 1
                if name == 'layer3.' + str(bottleneck_index) + '.' + 'conv3.weight':
                    bottleneck_index = bottleneck_index + 1
                    conv_index = 1
                elif name == 'layer4.' + str(bottleneck_index) + '.' + 'conv3.weight':
                    bottleneck_index = bottleneck_index + 1
                    conv_index = 1
        children_index = children_index + 1
        children_name = 'layer'+str(children_index)
        bottleneck_index = 0
'''        




#increase fine-tune layer per epoch
'''
if epoch < 6:
    # freeze backbone layers
    for param in net.parameters():
        count +=1
        if count < 4: #freezing first 3 layers
            param.requires_grad = False    
        else:
            for param in net.parameters():
                aram.requires_grad = True
'''

Training dataset size: 10950
Validation dataset size: 300
['false', 'pneumonia']
######### Validation Dataset #########
false size: 200
pneumonia size: 100
layer4의 2번째-bottleneck의 bn1.weight
layer4.2.bn1.weight의 gammm one setting 완료

layer4의 2번째-bottleneck의 bn1.bias
layer4.2.bn1.bias의 beta zero setting 완료

layer4의 2번째-bottleneck의 bn2.weight
layer4.2.bn2.weight의 gammm one setting 완료

layer4의 2번째-bottleneck의 bn2.bias
layer4.2.bn2.bias의 beta zero setting 완료

layer4의 2번째-bottleneck의 bn3.weight
layer4.2.bn3.weight의 gammm zero setting 완료

layer4의 2번째-bottleneck의 bn3.bias
layer4.2.bn3.bias의 beta zero setting 완료

Epoch 0/19
----------
val Loss: 0.7194 Acc: 35.6667

데이터셋A: 300
False Recall: 6.5
False Precision: 68.42
False ACC: 35.67
False F1 score: 11.87

데이터셋B: 300
Pneumonia Recall: 94.0
Pneumonia Precision: 33.45
Pneumonia ACC: 35.67
Pneumonia F1 score: 49.34


Epoch 1/19
----------
val Loss: 0.4808 Acc: 76.6667

데이터셋A: 300
False Recall: 69.0
False Precision: 94.52000000000001
False ACC: 76.

'\nif epoch < 6:\n    # freeze backbone layers\n    for param in net.parameters():\n        count +=1\n        if count < 4: #freezing first 3 layers\n            param.requires_grad = False    \n        else:\n            for param in net.parameters():\n                aram.requires_grad = True\n'

# seed 2  Warm up + Batch norm gamma setting (layer 4 bottleneck index 2)

In [3]:
# Author: Sasank Chilamkurthy

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import numpy as np
from openpyxl import Workbook
from warmup_scheduler import GradualWarmupScheduler
# from pytorch_cosine_annealing_warmup_master import cosine_annearing_with_warmup
import random


plt.ion()   # interactive mode

#torch.set_deterministic(True)
random_seed = 2
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.ColorJitter(brightness=(0.8, 1.2), contrast=(0.8, 1.3)),
        transforms.RandomAffine(degrees=0, translate=(0.15,0.1)),
        transforms.RandomAffine(degrees=(-10,10)),
        transforms.Resize(280),
        transforms.CenterCrop(224),
        #transforms.Resize(size=(224,224)),
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        transforms.Normalize([0.493, 0.493, 0.493], [0.246, 0.246, 0.246])
    ]),
    'val': transforms.Compose([
        #transforms.Resize(size=(224,224)),
        transforms.Resize(280),
        transforms.CenterCrop(224),
        #transforms.Resize(size=(224,224)),
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        transforms.Normalize([0.493, 0.493, 0.493], [0.246, 0.246, 0.246])
    ]),
}

data_dir = '/home/aiffel/Hackathon_covid_19/data_2'
#data_dir = 'Crop_CXR/binary_covid'
batch_size = 16
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                             shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
val_class_names = image_datasets['val'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print("Training dataset size: " + str(dataset_sizes['train']))
print("Validation dataset size: " +str(dataset_sizes['val']))
print(val_class_names)


val_dir = '/home/aiffel/Hackathon_covid_19/data_2/val/'
false_data_dir = val_dir + 'false'
pneumonia_data_dir = val_dir + 'pneumonia'

print("######### Validation Dataset #########")
val_false_num = len(os.listdir(false_data_dir))
val_pneumonia_num = len(os.listdir(pneumonia_data_dir))
print("false size: " + str(val_false_num))
print("pneumonia size: " + str(val_pneumonia_num))

wb = Workbook()      # 워크북을 생성한다.
ws = wb.active       # 워크 시트를 얻는다.
    
ws['A1'] = 'ResNet50'
ws['B1'] = 'Val ACC'

ws['D1'] = 'False ACC'
ws['E1'] = 'False Recall'
ws['F1'] = 'False Precision'
ws['G1'] = 'False F1'

ws['I1'] = 'Penumonia ACC'
ws['J1'] = 'Penumonia Recall'
ws['K1'] = 'Penumonia Precision'
ws['L1'] = 'Penumonia F1'


def train_model(model, criterion, optimizer, scheduler, num_epochs):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    best_false_acc = 0.0
    best_false_f1 = 0.0
    best_pneumonia_acc = 0.0
    best_pneumonia_f1 = 0.0

    best_false_acc_epoch = 0
    best_false_f1_epoch = 0
    best_pneumonia_acc_epoch = 0
    best_pneumonia_f1_epoch = 0

    for epoch in range(num_epochs):
        val_false_TP = 0.0
        val_pneumonia_TP = 0.0
        val_false_FN = 0.0
        val_pneumonia_FN = 0.0
        val_false_TN = 0.0
        val_pneumonia_TN = 0.0
        val_false_FP = 0.0
        val_pneumonia_FP = 0.0

        
        
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        A = 'A'+str(epoch+2)
        ws[A] = 'Epoch' + str(epoch+1)
    
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    #outputs = torch.nn.functional.softmax(outputs, dim=1)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    # Append batch prediction results

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
                a = preds.size()
                b = a[0]
                
                if phase == 'val':
                    for i in range(b):
                        if preds[i].item() == labels.data[i].item():
                            #Normal-index:0, Pneumonia-index:1
                            if preds.data[i].item() == 0: 
                                val_false_TP += 1 #Normal 관점에서는 normal를 정확히 분류하는 것이 TP
                                val_pneumonia_TN += 1 #Pneumonia 관점에서는 normal를 정확히 분류하는 것이 TN
                            elif preds.data[i].item() == 1: 
                                val_pneumonia_TP += 1 #Pneumonia 관점에서는 Pneumonia를 정확히 분류하는 것이 TP
                                val_false_TN += 1 #Normal 관점에서는 Pneumonia를 정확히 분류하는 것이 TN

                        if preds[i].item() != labels.data[i].item():
                            if preds.data[i].item() == 0:
                                val_false_FP += 1 #Normal 관점에서 normal라고 분류했지만 실제로는 pneumonia인 경우는 FP
                                val_pneumonia_FN += 1 #Pneumonia 관점에서 normal라고 분류했지만 실제로는 pneumonia인 경우는 FN
                            elif preds.data[i].item() == 1:
                                val_pneumonia_FP += 1 #Normal 관점에서 pneumonia라고 분류했지만 실제로는 normal인 경우는 FP
                                val_false_FN += 1 #Pneumonia 관점에서 pneumonia라고 분류했지만 실제로는 normal인 경우는 FN

            if phase == 'train':
                scheduler.step()
                
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
           
            if phase == 'val':
                B = 'B'+str(epoch+2)
                ws[B] = '{:.4f}'.format(epoch_acc*100)

                print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, (epoch_acc*100)))    

            
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            

        #recall, precision -> https://en.wikipedia.org/wiki/Precision_and_recall
        #recall = TP/(TP+FN)
        #precision = TP/(TP+FP)
        #ACC = (TP+TN)/(TP+TN+FP+FN)
        false_recall = round(val_false_TP/(val_false_TP + val_false_FN),4)*100
        pneumonia_recall = round(val_pneumonia_TP/(val_pneumonia_TP + val_pneumonia_FN),4)*100
        false_precision = round(val_false_TP/(val_false_TP + val_false_FP),4)*100
        pneumonia_precision = round(val_pneumonia_TP/(val_pneumonia_TP + val_pneumonia_FP),4)*100
        false_acc = round((val_false_TP + val_false_TN)/(val_false_TP + val_false_FP +val_false_TN + val_false_FN),4)*100
        pneumonia_acc = round((val_pneumonia_TP + val_pneumonia_TN)/(val_pneumonia_TP + val_pneumonia_FP +val_pneumonia_TN + val_pneumonia_FN),4)*100
        #F1 score -> https://en.wikipedia.org/wiki/F-score
        #F1 score = 2/((1/recall)+(1/precision)) = 2((precision*recall)/(precision+recall)) = tp/(tp+((1/2)(fp+fn)))
        false_f1_score = round(2*(false_precision*false_recall)/(false_precision+false_recall),2)
        pneumonia_f1_score = round(2*(pneumonia_precision*pneumonia_recall)/(pneumonia_precision+pneumonia_recall),2)

        #total_data = batch size --> covid_total_dataset = false_total_dataset
        false_total_dataset = int(val_false_TP + val_false_FP +val_false_TN + val_false_FN)
        pneumonia_total_dataset = int(val_pneumonia_TP + val_pneumonia_FP +val_pneumonia_TN + val_pneumonia_FN)
        
      
        D = 'D'+str(epoch+2)
        E = 'E'+str(epoch+2)
        F = 'F'+str(epoch+2)
        G = 'G'+str(epoch+2)
        
        I = 'I'+str(epoch+2)
        J = 'J'+str(epoch+2)
        K = 'K'+str(epoch+2)
        L = 'L'+str(epoch+2)

        ws[D] = false_acc
        ws[E] = false_recall
        ws[F] = false_precision
        ws[G] = false_f1_score
        
        ws[I] = pneumonia_acc
        ws[J] = pneumonia_recall
        ws[K] = pneumonia_precision
        ws[L] = pneumonia_f1_score


        print()

        print('데이터셋A: ' + str(false_total_dataset))
        print('False Recall: ' + str(false_recall))        
        print('False Precision: ' + str(false_precision))
        print('False ACC: ' + str(false_acc))
        print('False F1 score: ' + str(false_f1_score))
        
        print()
        
        print('데이터셋B: ' + str(pneumonia_total_dataset))
        print('Pneumonia Recall: ' + str(pneumonia_recall))
        print('Pneumonia Precision: ' + str(pneumonia_precision))
        print('Pneumonia ACC: ' + str(pneumonia_acc))
        print('Pneumonia F1 score: ' + str(pneumonia_f1_score))
        
  
        if phase == 'val' and false_acc > best_false_acc:
            best_false_acc = false_acc
            best_false_acc_epoch = epoch

        if phase == 'val' and pneumonia_acc > best_pneumonia_acc:
            best_pneumonia_acc = pneumonia_acc
            best_pneumonia_acc_epoch = epoch
            

        if phase == 'val' and false_f1_score > best_false_f1:
            best_false_f1 = false_f1_score
            best_false_f1_epoch = epoch
            
        if phase == 'val' and pneumonia_f1_score > best_pneumonia_f1:
            best_pneumonia_f1 = pneumonia_f1_score
            best_pneumonia_f1_epoch = epoch

        print()
        print()

    
    ws['N1'] = 'Best False ACC'
    ws['O1'] = 'epoch'
    ws['N2'] = round(best_false_acc,2)
    ws['O2'] = best_false_acc_epoch+1
        
    ws['N4'] = 'Best False F1'
    ws['O4'] = 'epoch'
    ws['N5'] = round(best_false_f1,2)
    ws['O5'] = best_false_f1_epoch+1
        
    ws['N7'] = 'Best Pneumonia ACC'
    ws['O7'] = 'epoch'
    ws['N8'] = round(best_pneumonia_acc,2)
    ws['O8'] = best_pneumonia_acc_epoch+1
        
    ws['N10'] = 'Best Pneumonia F1'
    ws['O10'] = 'epoch'
    ws['N11'] = round(best_pneumonia_f1,2)
    ws['O11'] = best_pneumonia_f1_epoch+1

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    #print('Best val Acc: {:4f}'.format(best_acc))


   

    # load best model weights
    #model.load_state_dict(best_covid_wts)
    wb.save('output_excel/현수님세팅_Pneumonia_vs_Fasle(train5475_P 기준세팅)_Balanced(val)_seed_2_BatchNorm_layer_4_bottleneck_2.xlsx') # 엑셀로 저장한다. 
    #torch.save(model.state_dict(), 'covid_binary.pt')
    return model


model_ft = models.resnet50(pretrained=True)

###BatchNorm Gamma setting###


# seed 0,1,2

# count =0
# bottleneck_count = 0
# layer_index=1
# layer_name = 'layer'+str(layer_index)
# bottleneck_index = 0



# count =0
# bottleneck_count = 0
# layer_index=4
# layer_name = 'layer'+str(layer_index)
# bottleneck_index = 0


count =0
bottleneck_count = 0
layer_index=4
layer_name = 'layer'+str(layer_index)
bottleneck_index = 2



for name, layer in model_ft.named_children():
    if bottleneck_count > 0:
        bottleneck_index = 0
    if name == layer_name:
        bn_index = 1
        for name, param in model_ft.named_parameters():
            if name == layer_name + '.' + str(bottleneck_index) + '.' + 'bn1.weight':
                print(str(layer_name + "의 " + str(bottleneck_index)+ "번째-bottleneck의 " + 'bn1.weight'))
                torch.nn.init.ones_(param)
                print(name + '의 gammm one setting 완료')
                print()
                
            elif name == layer_name + '.' + str(bottleneck_index) + '.' + 'bn2.weight':
                print(str(layer_name + "의 " + str(bottleneck_index)+ "번째-bottleneck의 " + 'bn2.weight'))
                torch.nn.init.ones_(param)
                print(name + '의 gammm one setting 완료')
                print()

            elif name == layer_name + '.' + str(bottleneck_index) + '.' + 'bn3.weight':
                print(str(layer_name + "의 " + str(bottleneck_index)+ "번째-bottleneck의 " + 'bn3.weight'))
                torch.nn.init.zeros_(param)
                print(name + '의 gammm zero setting 완료')
                print()

            elif name == layer_name + '.' + str(bottleneck_index) + '.' + 'bn' + str(bn_index) + '.bias':
                print(str(layer_name + "의 " + str(bottleneck_index)+ "번째-bottleneck의 " + 'bn' + str(bn_index) + '.bias'))
                torch.nn.init.zeros_(param)
                bn_index = bn_index + 1
                if name == layer_name + '.' + str(bottleneck_index) + '.' + 'bn3.bias':
                    bottleneck_index = bottleneck_index + 1
                    bn_index = 1
                print(name + '의 beta zero setting 완료')
                print()

        bottleneck_count = bottleneck_count + 1
        layer_index = layer_index+ 1
        layer_name = 'layer'+str(layer_index)     

for param in model_ft.parameters():
    param.requires_grad = True

num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 2)




model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=3, gamma=0.1)
#exp_lr_scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max, eta_min=0, last_epoch=-1, verbose=False)
exp_lr_scheduler = GradualWarmupScheduler(optimizer_ft, multiplier=1, total_epoch=5, after_scheduler=exp_lr_scheduler)
#exp_lr_scheduler = cosine_annearing_with_warmup.CosineAnnealingWarmUpRestarts(optimizer_ft, T_0=30, T_mult=1, eta_max=0.001, T_up=20)

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=20)

# freeze backbone layers
'''
count = 0
for child_name, param in model_ft.named_children():
    for param_name, param_param in model_ft.named_parameters():
        if child_name == 'layer4': 
            param_param.requires_grad = True
        else:
            param_param.requires_grad = False
'''


#특정 conv layer 이후 초기화
'''
children_index = 4
children_name = 'layer'+str(children_index)
bottleneck_index = 2
for name, layer in model_ft.named_children():
    if name == children_name:
        conv_index = 1
        for name, param in model_ft.named_parameters():
            if name == children_name + "." + str(bottleneck_index) + '.' + 'conv' + str(conv_index) + '.weight':
                print(children_name + "의 "+ str(bottleneck_index)+ "번째-bottleneck의 " + 'conv' +str(conv_index) + '.weight')
                torch.nn.init.xavier_uniform_(param)
                print(name + '의 conv filter initilization setting 완료')
                print()
                conv_index = conv_index + 1
                if name == 'layer3.' + str(bottleneck_index) + '.' + 'conv3.weight':
                    bottleneck_index = bottleneck_index + 1
                    conv_index = 1
                elif name == 'layer4.' + str(bottleneck_index) + '.' + 'conv3.weight':
                    bottleneck_index = bottleneck_index + 1
                    conv_index = 1
        children_index = children_index + 1
        children_name = 'layer'+str(children_index)
        bottleneck_index = 0
'''        




#increase fine-tune layer per epoch
'''
if epoch < 6:
    # freeze backbone layers
    for param in net.parameters():
        count +=1
        if count < 4: #freezing first 3 layers
            param.requires_grad = False    
        else:
            for param in net.parameters():
                aram.requires_grad = True
'''

Training dataset size: 10950
Validation dataset size: 300
['false', 'pneumonia']
######### Validation Dataset #########
false size: 200
pneumonia size: 100
layer4의 2번째-bottleneck의 bn1.weight
layer4.2.bn1.weight의 gammm one setting 완료

layer4의 2번째-bottleneck의 bn1.bias
layer4.2.bn1.bias의 beta zero setting 완료

layer4의 2번째-bottleneck의 bn2.weight
layer4.2.bn2.weight의 gammm one setting 완료

layer4의 2번째-bottleneck의 bn2.bias
layer4.2.bn2.bias의 beta zero setting 완료

layer4의 2번째-bottleneck의 bn3.weight
layer4.2.bn3.weight의 gammm zero setting 완료

layer4의 2번째-bottleneck의 bn3.bias
layer4.2.bn3.bias의 beta zero setting 완료

Epoch 0/19
----------
val Loss: 0.7225 Acc: 29.6667

데이터셋A: 300
False Recall: 19.0
False Precision: 43.68
False ACC: 29.67
False F1 score: 26.48

데이터셋B: 300
Pneumonia Recall: 51.0
Pneumonia Precision: 23.94
Pneumonia ACC: 29.67
Pneumonia F1 score: 32.58


Epoch 1/19
----------
val Loss: 0.4928 Acc: 75.6667

데이터셋A: 300
False Recall: 68.5
False Precision: 93.2
False ACC: 75.67
False F1 

'\nif epoch < 6:\n    # freeze backbone layers\n    for param in net.parameters():\n        count +=1\n        if count < 4: #freezing first 3 layers\n            param.requires_grad = False    \n        else:\n            for param in net.parameters():\n                aram.requires_grad = True\n'

# Seed 0 _  Mistake code _ layer 4 bottindex 0 

In [2]:
# Author: Sasank Chilamkurthy

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import numpy as np
from openpyxl import Workbook
from warmup_scheduler import GradualWarmupScheduler
# from pytorch_cosine_annealing_warmup_master import cosine_annearing_with_warmup
import random


plt.ion()   # interactive mode

#torch.set_deterministic(True)
random_seed = 0
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.ColorJitter(brightness=(0.8, 1.2), contrast=(0.8, 1.3)),
        transforms.RandomAffine(degrees=0, translate=(0.15,0.1)),
        transforms.RandomAffine(degrees=(-10,10)),
        transforms.Resize(280),
        transforms.CenterCrop(224),
        #transforms.Resize(size=(224,224)),
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        transforms.Normalize([0.493, 0.493, 0.493], [0.246, 0.246, 0.246])
    ]),
    'val': transforms.Compose([
        #transforms.Resize(size=(224,224)),
        transforms.Resize(280),
        transforms.CenterCrop(224),
        #transforms.Resize(size=(224,224)),
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        transforms.Normalize([0.493, 0.493, 0.493], [0.246, 0.246, 0.246])
    ]),
}

data_dir = '/home/aiffel/Hackathon_covid_19/data_2'
#data_dir = 'Crop_CXR/binary_covid'
batch_size = 16
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                             shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
val_class_names = image_datasets['val'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print("Training dataset size: " + str(dataset_sizes['train']))
print("Validation dataset size: " +str(dataset_sizes['val']))
print(val_class_names)


val_dir = '/home/aiffel/Hackathon_covid_19/data_2/val/'
false_data_dir = val_dir + 'false'
pneumonia_data_dir = val_dir + 'pneumonia'

print("######### Validation Dataset #########")
val_false_num = len(os.listdir(false_data_dir))
val_pneumonia_num = len(os.listdir(pneumonia_data_dir))
print("false size: " + str(val_false_num))
print("pneumonia size: " + str(val_pneumonia_num))

wb = Workbook()      # 워크북을 생성한다.
ws = wb.active       # 워크 시트를 얻는다.
    
ws['A1'] = 'ResNet50'
ws['B1'] = 'Val ACC'

ws['D1'] = 'False ACC'
ws['E1'] = 'False Recall'
ws['F1'] = 'False Precision'
ws['G1'] = 'False F1'

ws['I1'] = 'Penumonia ACC'
ws['J1'] = 'Penumonia Recall'
ws['K1'] = 'Penumonia Precision'
ws['L1'] = 'Penumonia F1'


def train_model(model, criterion, optimizer, scheduler, num_epochs):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    best_false_acc = 0.0
    best_false_f1 = 0.0
    best_pneumonia_acc = 0.0
    best_pneumonia_f1 = 0.0

    best_false_acc_epoch = 0
    best_false_f1_epoch = 0
    best_pneumonia_acc_epoch = 0
    best_pneumonia_f1_epoch = 0

    for epoch in range(num_epochs):
        val_false_TP = 0.0
        val_pneumonia_TP = 0.0
        val_false_FN = 0.0
        val_pneumonia_FN = 0.0
        val_false_TN = 0.0
        val_pneumonia_TN = 0.0
        val_false_FP = 0.0
        val_pneumonia_FP = 0.0

        
        
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        A = 'A'+str(epoch+2)
        ws[A] = 'Epoch' + str(epoch+1)
    
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    #outputs = torch.nn.functional.softmax(outputs, dim=1)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    # Append batch prediction results

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
                a = preds.size()
                b = a[0]
                
                if phase == 'val':
                    for i in range(b):
                        if preds[i].item() == labels.data[i].item():
                            #Normal-index:0, Pneumonia-index:1
                            if preds.data[i].item() == 0: 
                                val_false_TP += 1 #Normal 관점에서는 normal를 정확히 분류하는 것이 TP
                                val_pneumonia_TN += 1 #Pneumonia 관점에서는 normal를 정확히 분류하는 것이 TN
                            elif preds.data[i].item() == 1: 
                                val_pneumonia_TP += 1 #Pneumonia 관점에서는 Pneumonia를 정확히 분류하는 것이 TP
                                val_false_TN += 1 #Normal 관점에서는 Pneumonia를 정확히 분류하는 것이 TN

                        if preds[i].item() != labels.data[i].item():
                            if preds.data[i].item() == 0:
                                val_false_FP += 1 #Normal 관점에서 normal라고 분류했지만 실제로는 pneumonia인 경우는 FP
                                val_pneumonia_FN += 1 #Pneumonia 관점에서 normal라고 분류했지만 실제로는 pneumonia인 경우는 FN
                            elif preds.data[i].item() == 1:
                                val_pneumonia_FP += 1 #Normal 관점에서 pneumonia라고 분류했지만 실제로는 normal인 경우는 FP
                                val_false_FN += 1 #Pneumonia 관점에서 pneumonia라고 분류했지만 실제로는 normal인 경우는 FN

            if phase == 'train':
                scheduler.step()
                
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
           
            if phase == 'val':
                B = 'B'+str(epoch+2)
                ws[B] = '{:.4f}'.format(epoch_acc*100)

                print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, (epoch_acc*100)))    

            
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            

        #recall, precision -> https://en.wikipedia.org/wiki/Precision_and_recall
        #recall = TP/(TP+FN)
        #precision = TP/(TP+FP)
        #ACC = (TP+TN)/(TP+TN+FP+FN)
        false_recall = round(val_false_TP/(val_false_TP + val_false_FN),4)*100
        pneumonia_recall = round(val_pneumonia_TP/(val_pneumonia_TP + val_pneumonia_FN),4)*100
        false_precision = round(val_false_TP/(val_false_TP + val_false_FP),4)*100
        pneumonia_precision = round(val_pneumonia_TP/(val_pneumonia_TP + val_pneumonia_FP),4)*100
        false_acc = round((val_false_TP + val_false_TN)/(val_false_TP + val_false_FP +val_false_TN + val_false_FN),4)*100
        pneumonia_acc = round((val_pneumonia_TP + val_pneumonia_TN)/(val_pneumonia_TP + val_pneumonia_FP +val_pneumonia_TN + val_pneumonia_FN),4)*100
        #F1 score -> https://en.wikipedia.org/wiki/F-score
        #F1 score = 2/((1/recall)+(1/precision)) = 2((precision*recall)/(precision+recall)) = tp/(tp+((1/2)(fp+fn)))
        false_f1_score = round(2*(false_precision*false_recall)/(false_precision+false_recall),2)
        pneumonia_f1_score = round(2*(pneumonia_precision*pneumonia_recall)/(pneumonia_precision+pneumonia_recall),2)

        #total_data = batch size --> covid_total_dataset = false_total_dataset
        false_total_dataset = int(val_false_TP + val_false_FP +val_false_TN + val_false_FN)
        pneumonia_total_dataset = int(val_pneumonia_TP + val_pneumonia_FP +val_pneumonia_TN + val_pneumonia_FN)
        
      
        D = 'D'+str(epoch+2)
        E = 'E'+str(epoch+2)
        F = 'F'+str(epoch+2)
        G = 'G'+str(epoch+2)
        
        I = 'I'+str(epoch+2)
        J = 'J'+str(epoch+2)
        K = 'K'+str(epoch+2)
        L = 'L'+str(epoch+2)

        ws[D] = false_acc
        ws[E] = false_recall
        ws[F] = false_precision
        ws[G] = false_f1_score
        
        ws[I] = pneumonia_acc
        ws[J] = pneumonia_recall
        ws[K] = pneumonia_precision
        ws[L] = pneumonia_f1_score


        print()

        print('데이터셋A: ' + str(false_total_dataset))
        print('False Recall: ' + str(false_recall))        
        print('False Precision: ' + str(false_precision))
        print('False ACC: ' + str(false_acc))
        print('False F1 score: ' + str(false_f1_score))
        
        print()
        
        print('데이터셋B: ' + str(pneumonia_total_dataset))
        print('Pneumonia Recall: ' + str(pneumonia_recall))
        print('Pneumonia Precision: ' + str(pneumonia_precision))
        print('Pneumonia ACC: ' + str(pneumonia_acc))
        print('Pneumonia F1 score: ' + str(pneumonia_f1_score))
        
  
        if phase == 'val' and false_acc > best_false_acc:
            best_false_acc = false_acc
            best_false_acc_epoch = epoch

        if phase == 'val' and pneumonia_acc > best_pneumonia_acc:
            best_pneumonia_acc = pneumonia_acc
            best_pneumonia_acc_epoch = epoch
            

        if phase == 'val' and false_f1_score > best_false_f1:
            best_false_f1 = false_f1_score
            best_false_f1_epoch = epoch
            
        if phase == 'val' and pneumonia_f1_score > best_pneumonia_f1:
            best_pneumonia_f1 = pneumonia_f1_score
            best_pneumonia_f1_epoch = epoch

        print()
        print()

    
    ws['N1'] = 'Best False ACC'
    ws['O1'] = 'epoch'
    ws['N2'] = round(best_false_acc,2)
    ws['O2'] = best_false_acc_epoch+1
        
    ws['N4'] = 'Best False F1'
    ws['O4'] = 'epoch'
    ws['N5'] = round(best_false_f1,2)
    ws['O5'] = best_false_f1_epoch+1
        
    ws['N7'] = 'Best Pneumonia ACC'
    ws['O7'] = 'epoch'
    ws['N8'] = round(best_pneumonia_acc,2)
    ws['O8'] = best_pneumonia_acc_epoch+1
        
    ws['N10'] = 'Best Pneumonia F1'
    ws['O10'] = 'epoch'
    ws['N11'] = round(best_pneumonia_f1,2)
    ws['O11'] = best_pneumonia_f1_epoch+1

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    #print('Best val Acc: {:4f}'.format(best_acc))


   

    # load best model weights
    #model.load_state_dict(best_covid_wts)
    wb.save('output_excel/현수님세팅_Pneumonia_vs_Fasle(train5475_P 기준세팅)_Balanced(val)_seed_0_Mistake_layer_4_bottleneck_0.xlsx') # 엑셀로 저장한다. 
    #torch.save(model.state_dict(), 'covid_binary.pt')
    return model


model_ft = models.resnet50(pretrained=True)


###BatchNorm Gamma setting###

count =0
layer_index=4
layer_name = 'layer'+str(layer_index)

for name, layer in model_ft.named_children():
    if name == layer_name:
        bottleneck_index = 0
        bn_index = 1
        for name, param in model_ft.named_parameters():
            if name == layer_name + '.' + str(bottleneck_index) + '.' + 'bn1.weight':
                print(str(layer_name + "의 " + str(bottleneck_index)+ "번째-bottleneck의 " + 'bn1.weight'))
                torch.nn.init.ones_(param)
                print(name + '의 gammm one setting 완료')
                print()
                
            elif name == layer_name + '.' + str(bottleneck_index) + '.' + 'bn2.weight':
                print(str(layer_name + "의 " + str(bottleneck_index)+ "번째-bottleneck의 " + 'bn2.weight'))
                torch.nn.init.ones_(param)
                print(name + '의 gammm one setting 완료')
                print()

            elif name == layer_name + '.' + str(bottleneck_index) + '.' + 'bn3.weight':
                print(str(layer_name + "의 " + str(bottleneck_index)+ "번째-bottleneck의 " + 'bn3.weight'))
                torch.nn.init.ones_(param)
                print(name + '의 gammm zero setting 완료')
                print()

            elif name == layer_name + '.' + str(bottleneck_index) + '.' + 'bn' + str(bn_index) + '.bias':
                print(str(layer_name + "의 " + str(bottleneck_index)+ "번째-bottleneck의 " + 'bn' + str(bn_index) + '.bias'))
                torch.nn.init.zeros_(param)
                bn_index = bn_index + 1
                print(name + '의 beta zero setting 완료')
                print()



        bottleneck_index = bottleneck_index + 1

        layer_index = layer_index+ 1
        layer_name = 'layer'+str(layer_index)     

for param in model_ft.parameters():
    param.requires_grad = True

num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 2)




model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=3, gamma=0.1)
#exp_lr_scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max, eta_min=0, last_epoch=-1, verbose=False)
exp_lr_scheduler = GradualWarmupScheduler(optimizer_ft, multiplier=1, total_epoch=5, after_scheduler=exp_lr_scheduler)
#exp_lr_scheduler = cosine_annearing_with_warmup.CosineAnnealingWarmUpRestarts(optimizer_ft, T_0=30, T_mult=1, eta_max=0.001, T_up=20)

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=20)

# freeze backbone layers
'''
count = 0
for child_name, param in model_ft.named_children():
    for param_name, param_param in model_ft.named_parameters():
        if child_name == 'layer4': 
            param_param.requires_grad = True
        else:
            param_param.requires_grad = False
'''


#특정 conv layer 이후 초기화
'''
children_index = 4
children_name = 'layer'+str(children_index)
bottleneck_index = 2
for name, layer in model_ft.named_children():
    if name == children_name:
        conv_index = 1
        for name, param in model_ft.named_parameters():
            if name == children_name + "." + str(bottleneck_index) + '.' + 'conv' + str(conv_index) + '.weight':
                print(children_name + "의 "+ str(bottleneck_index)+ "번째-bottleneck의 " + 'conv' +str(conv_index) + '.weight')
                torch.nn.init.xavier_uniform_(param)
                print(name + '의 conv filter initilization setting 완료')
                print()
                conv_index = conv_index + 1
                if name == 'layer3.' + str(bottleneck_index) + '.' + 'conv3.weight':
                    bottleneck_index = bottleneck_index + 1
                    conv_index = 1
                elif name == 'layer4.' + str(bottleneck_index) + '.' + 'conv3.weight':
                    bottleneck_index = bottleneck_index + 1
                    conv_index = 1
        children_index = children_index + 1
        children_name = 'layer'+str(children_index)
        bottleneck_index = 0
'''        




#increase fine-tune layer per epoch
'''
if epoch < 6:
    # freeze backbone layers
    for param in net.parameters():
        count +=1
        if count < 4: #freezing first 3 layers
            param.requires_grad = False    
        else:
            for param in net.parameters():
                aram.requires_grad = True
'''

Training dataset size: 10950
Validation dataset size: 300
['false', 'pneumonia']
######### Validation Dataset #########
false size: 200
pneumonia size: 100
layer4의 0번째-bottleneck의 bn1.weight
layer4.0.bn1.weight의 gammm one setting 완료

layer4의 0번째-bottleneck의 bn1.bias
layer4.0.bn1.bias의 beta zero setting 완료

layer4의 0번째-bottleneck의 bn2.weight
layer4.0.bn2.weight의 gammm one setting 완료

layer4의 0번째-bottleneck의 bn2.bias
layer4.0.bn2.bias의 beta zero setting 완료

layer4의 0번째-bottleneck의 bn3.weight
layer4.0.bn3.weight의 gammm zero setting 완료

layer4의 0번째-bottleneck의 bn3.bias
layer4.0.bn3.bias의 beta zero setting 완료

Epoch 0/19
----------
val Loss: 0.6366 Acc: 65.3333

데이터셋A: 300
False Recall: 92.0
False Precision: 67.65
False ACC: 65.33
False F1 score: 77.97

데이터셋B: 300
Pneumonia Recall: 12.0
Pneumonia Precision: 42.86
Pneumonia ACC: 65.33
Pneumonia F1 score: 18.75


Epoch 1/19
----------
val Loss: 0.4134 Acc: 82.0000

데이터셋A: 300
False Recall: 76.0
False Precision: 96.2
False ACC: 82.0
False F1 s

'\nif epoch < 6:\n    # freeze backbone layers\n    for param in net.parameters():\n        count +=1\n        if count < 4: #freezing first 3 layers\n            param.requires_grad = False    \n        else:\n            for param in net.parameters():\n                aram.requires_grad = True\n'

# Seed 1 _  Mistake code _ layer 4 bottindex 0 

In [3]:
# Author: Sasank Chilamkurthy

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import numpy as np
from openpyxl import Workbook
from warmup_scheduler import GradualWarmupScheduler
# from pytorch_cosine_annealing_warmup_master import cosine_annearing_with_warmup
import random


plt.ion()   # interactive mode

#torch.set_deterministic(True)
random_seed = 1
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.ColorJitter(brightness=(0.8, 1.2), contrast=(0.8, 1.3)),
        transforms.RandomAffine(degrees=0, translate=(0.15,0.1)),
        transforms.RandomAffine(degrees=(-10,10)),
        transforms.Resize(280),
        transforms.CenterCrop(224),
        #transforms.Resize(size=(224,224)),
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        transforms.Normalize([0.493, 0.493, 0.493], [0.246, 0.246, 0.246])
    ]),
    'val': transforms.Compose([
        #transforms.Resize(size=(224,224)),
        transforms.Resize(280),
        transforms.CenterCrop(224),
        #transforms.Resize(size=(224,224)),
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        transforms.Normalize([0.493, 0.493, 0.493], [0.246, 0.246, 0.246])
    ]),
}

data_dir = '/home/aiffel/Hackathon_covid_19/data_2'
#data_dir = 'Crop_CXR/binary_covid'
batch_size = 16
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                             shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
val_class_names = image_datasets['val'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print("Training dataset size: " + str(dataset_sizes['train']))
print("Validation dataset size: " +str(dataset_sizes['val']))
print(val_class_names)


val_dir = '/home/aiffel/Hackathon_covid_19/data_2/val/'
false_data_dir = val_dir + 'false'
pneumonia_data_dir = val_dir + 'pneumonia'

print("######### Validation Dataset #########")
val_false_num = len(os.listdir(false_data_dir))
val_pneumonia_num = len(os.listdir(pneumonia_data_dir))
print("false size: " + str(val_false_num))
print("pneumonia size: " + str(val_pneumonia_num))

wb = Workbook()      # 워크북을 생성한다.
ws = wb.active       # 워크 시트를 얻는다.
    
ws['A1'] = 'ResNet50'
ws['B1'] = 'Val ACC'

ws['D1'] = 'False ACC'
ws['E1'] = 'False Recall'
ws['F1'] = 'False Precision'
ws['G1'] = 'False F1'

ws['I1'] = 'Penumonia ACC'
ws['J1'] = 'Penumonia Recall'
ws['K1'] = 'Penumonia Precision'
ws['L1'] = 'Penumonia F1'


def train_model(model, criterion, optimizer, scheduler, num_epochs):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    best_false_acc = 0.0
    best_false_f1 = 0.0
    best_pneumonia_acc = 0.0
    best_pneumonia_f1 = 0.0

    best_false_acc_epoch = 0
    best_false_f1_epoch = 0
    best_pneumonia_acc_epoch = 0
    best_pneumonia_f1_epoch = 0

    for epoch in range(num_epochs):
        val_false_TP = 0.0
        val_pneumonia_TP = 0.0
        val_false_FN = 0.0
        val_pneumonia_FN = 0.0
        val_false_TN = 0.0
        val_pneumonia_TN = 0.0
        val_false_FP = 0.0
        val_pneumonia_FP = 0.0

        
        
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        A = 'A'+str(epoch+2)
        ws[A] = 'Epoch' + str(epoch+1)
    
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    #outputs = torch.nn.functional.softmax(outputs, dim=1)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    # Append batch prediction results

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
                a = preds.size()
                b = a[0]
                
                if phase == 'val':
                    for i in range(b):
                        if preds[i].item() == labels.data[i].item():
                            #Normal-index:0, Pneumonia-index:1
                            if preds.data[i].item() == 0: 
                                val_false_TP += 1 #Normal 관점에서는 normal를 정확히 분류하는 것이 TP
                                val_pneumonia_TN += 1 #Pneumonia 관점에서는 normal를 정확히 분류하는 것이 TN
                            elif preds.data[i].item() == 1: 
                                val_pneumonia_TP += 1 #Pneumonia 관점에서는 Pneumonia를 정확히 분류하는 것이 TP
                                val_false_TN += 1 #Normal 관점에서는 Pneumonia를 정확히 분류하는 것이 TN

                        if preds[i].item() != labels.data[i].item():
                            if preds.data[i].item() == 0:
                                val_false_FP += 1 #Normal 관점에서 normal라고 분류했지만 실제로는 pneumonia인 경우는 FP
                                val_pneumonia_FN += 1 #Pneumonia 관점에서 normal라고 분류했지만 실제로는 pneumonia인 경우는 FN
                            elif preds.data[i].item() == 1:
                                val_pneumonia_FP += 1 #Normal 관점에서 pneumonia라고 분류했지만 실제로는 normal인 경우는 FP
                                val_false_FN += 1 #Pneumonia 관점에서 pneumonia라고 분류했지만 실제로는 normal인 경우는 FN

            if phase == 'train':
                scheduler.step()
                
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
           
            if phase == 'val':
                B = 'B'+str(epoch+2)
                ws[B] = '{:.4f}'.format(epoch_acc*100)

                print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, (epoch_acc*100)))    

            
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
#                 best_model_wts = copy.deepcopy(model.state_dict())
            

        #recall, precision -> https://en.wikipedia.org/wiki/Precision_and_recall
        #recall = TP/(TP+FN)
        #precision = TP/(TP+FP)
        #ACC = (TP+TN)/(TP+TN+FP+FN)
        false_recall = round(val_false_TP/(val_false_TP + val_false_FN),4)*100
        pneumonia_recall = round(val_pneumonia_TP/(val_pneumonia_TP + val_pneumonia_FN),4)*100
        false_precision = round(val_false_TP/(val_false_TP + val_false_FP),4)*100
        pneumonia_precision = round(val_pneumonia_TP/(val_pneumonia_TP + val_pneumonia_FP),4)*100
        false_acc = round((val_false_TP + val_false_TN)/(val_false_TP + val_false_FP +val_false_TN + val_false_FN),4)*100
        pneumonia_acc = round((val_pneumonia_TP + val_pneumonia_TN)/(val_pneumonia_TP + val_pneumonia_FP +val_pneumonia_TN + val_pneumonia_FN),4)*100
        #F1 score -> https://en.wikipedia.org/wiki/F-score
        #F1 score = 2/((1/recall)+(1/precision)) = 2((precision*recall)/(precision+recall)) = tp/(tp+((1/2)(fp+fn)))
        false_f1_score = round(2*(false_precision*false_recall)/(false_precision+false_recall),2)
        pneumonia_f1_score = round(2*(pneumonia_precision*pneumonia_recall)/(pneumonia_precision+pneumonia_recall),2)

        #total_data = batch size --> covid_total_dataset = false_total_dataset
        false_total_dataset = int(val_false_TP + val_false_FP +val_false_TN + val_false_FN)
        pneumonia_total_dataset = int(val_pneumonia_TP + val_pneumonia_FP +val_pneumonia_TN + val_pneumonia_FN)
        
      
        D = 'D'+str(epoch+2)
        E = 'E'+str(epoch+2)
        F = 'F'+str(epoch+2)
        G = 'G'+str(epoch+2)
        
        I = 'I'+str(epoch+2)
        J = 'J'+str(epoch+2)
        K = 'K'+str(epoch+2)
        L = 'L'+str(epoch+2)

        ws[D] = false_acc
        ws[E] = false_recall
        ws[F] = false_precision
        ws[G] = false_f1_score
        
        ws[I] = pneumonia_acc
        ws[J] = pneumonia_recall
        ws[K] = pneumonia_precision
        ws[L] = pneumonia_f1_score


        print()

        print('데이터셋A: ' + str(false_total_dataset))
        print('False Recall: ' + str(false_recall))        
        print('False Precision: ' + str(false_precision))
        print('False ACC: ' + str(false_acc))
        print('False F1 score: ' + str(false_f1_score))
        
        print()
        
        print('데이터셋B: ' + str(pneumonia_total_dataset))
        print('Pneumonia Recall: ' + str(pneumonia_recall))
        print('Pneumonia Precision: ' + str(pneumonia_precision))
        print('Pneumonia ACC: ' + str(pneumonia_acc))
        print('Pneumonia F1 score: ' + str(pneumonia_f1_score))
        
  
        if phase == 'val' and false_acc > best_false_acc:
            best_false_acc = false_acc
            best_false_acc_epoch = epoch

        if phase == 'val' and pneumonia_acc > best_pneumonia_acc:
            best_pneumonia_acc = pneumonia_acc
            best_pneumonia_acc_epoch = epoch
            

        if phase == 'val' and false_f1_score > best_false_f1:
            best_false_f1 = false_f1_score
            best_false_f1_epoch = epoch
            
        if phase == 'val' and pneumonia_f1_score > best_pneumonia_f1:
            best_pneumonia_f1 = pneumonia_f1_score
            best_pneumonia_f1_epoch = epoch
#             best_model_wts = copy.deepcopy(model.state_dict())

        print()
        print()

    
    ws['N1'] = 'Best False ACC'
    ws['O1'] = 'epoch'
    ws['N2'] = round(best_false_acc,2)
    ws['O2'] = best_false_acc_epoch+1
        
    ws['N4'] = 'Best False F1'
    ws['O4'] = 'epoch'
    ws['N5'] = round(best_false_f1,2)
    ws['O5'] = best_false_f1_epoch+1
        
    ws['N7'] = 'Best Pneumonia ACC'
    ws['O7'] = 'epoch'
    ws['N8'] = round(best_pneumonia_acc,2)
    ws['O8'] = best_pneumonia_acc_epoch+1
        
    ws['N10'] = 'Best Pneumonia F1'
    ws['O10'] = 'epoch'
    ws['N11'] = round(best_pneumonia_f1,2)
    ws['O11'] = best_pneumonia_f1_epoch+1

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    #print('Best val Acc: {:4f}'.format(best_acc))


   

    # load best model weights
    #model.load_state_dict(best_covid_wts)
    wb.save('output_excel/현수님세팅_Pneumonia_vs_Fasle(train5475_P 기준세팅)_Balanced(val)_seed_1_Mistake_layer_4_bottleneck_0.xlsx') # 엑셀로 저장한다. 
    #torch.save(model.state_dict(), 'covid_binary.pt')
    return model


model_ft = models.resnet50(pretrained=True)


###BatchNorm Gamma setting###

count =0
layer_index=4
layer_name = 'layer'+str(layer_index)

for name, layer in model_ft.named_children():
    if name == layer_name:
        bottleneck_index = 0
        bn_index = 1
        for name, param in model_ft.named_parameters():
            if name == layer_name + '.' + str(bottleneck_index) + '.' + 'bn1.weight':
                print(str(layer_name + "의 " + str(bottleneck_index)+ "번째-bottleneck의 " + 'bn1.weight'))
                torch.nn.init.ones_(param)
                print(name + '의 gammm one setting 완료')
                print()
                
            elif name == layer_name + '.' + str(bottleneck_index) + '.' + 'bn2.weight':
                print(str(layer_name + "의 " + str(bottleneck_index)+ "번째-bottleneck의 " + 'bn2.weight'))
                torch.nn.init.ones_(param)
                print(name + '의 gammm one setting 완료')
                print()

            elif name == layer_name + '.' + str(bottleneck_index) + '.' + 'bn3.weight':
                print(str(layer_name + "의 " + str(bottleneck_index)+ "번째-bottleneck의 " + 'bn3.weight'))
                torch.nn.init.ones_(param)
                print(name + '의 gammm zero setting 완료')
                print()

            elif name == layer_name + '.' + str(bottleneck_index) + '.' + 'bn' + str(bn_index) + '.bias':
                print(str(layer_name + "의 " + str(bottleneck_index)+ "번째-bottleneck의 " + 'bn' + str(bn_index) + '.bias'))
                torch.nn.init.zeros_(param)
                bn_index = bn_index + 1
                print(name + '의 beta zero setting 완료')
                print()



        bottleneck_index = bottleneck_index + 1

        layer_index = layer_index+ 1
        layer_name = 'layer'+str(layer_index)     

for param in model_ft.parameters():
    param.requires_grad = True

num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 2)




model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=3, gamma=0.1)
#exp_lr_scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max, eta_min=0, last_epoch=-1, verbose=False)
exp_lr_scheduler = GradualWarmupScheduler(optimizer_ft, multiplier=1, total_epoch=5, after_scheduler=exp_lr_scheduler)
#exp_lr_scheduler = cosine_annearing_with_warmup.CosineAnnealingWarmUpRestarts(optimizer_ft, T_0=30, T_mult=1, eta_max=0.001, T_up=20)

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=20)

# freeze backbone layers
'''
count = 0
for child_name, param in model_ft.named_children():
    for param_name, param_param in model_ft.named_parameters():
        if child_name == 'layer4': 
            param_param.requires_grad = True
        else:
            param_param.requires_grad = False
'''


#특정 conv layer 이후 초기화
'''
children_index = 4
children_name = 'layer'+str(children_index)
bottleneck_index = 2
for name, layer in model_ft.named_children():
    if name == children_name:
        conv_index = 1
        for name, param in model_ft.named_parameters():
            if name == children_name + "." + str(bottleneck_index) + '.' + 'conv' + str(conv_index) + '.weight':
                print(children_name + "의 "+ str(bottleneck_index)+ "번째-bottleneck의 " + 'conv' +str(conv_index) + '.weight')
                torch.nn.init.xavier_uniform_(param)
                print(name + '의 conv filter initilization setting 완료')
                print()
                conv_index = conv_index + 1
                if name == 'layer3.' + str(bottleneck_index) + '.' + 'conv3.weight':
                    bottleneck_index = bottleneck_index + 1
                    conv_index = 1
                elif name == 'layer4.' + str(bottleneck_index) + '.' + 'conv3.weight':
                    bottleneck_index = bottleneck_index + 1
                    conv_index = 1
        children_index = children_index + 1
        children_name = 'layer'+str(children_index)
        bottleneck_index = 0
'''        




#increase fine-tune layer per epoch
'''
if epoch < 6:
    # freeze backbone layers
    for param in net.parameters():
        count +=1
        if count < 4: #freezing first 3 layers
            param.requires_grad = False    
        else:
            for param in net.parameters():
                aram.requires_grad = True
'''

Training dataset size: 10950
Validation dataset size: 300
['false', 'pneumonia']
######### Validation Dataset #########
false size: 200
pneumonia size: 100
layer4의 0번째-bottleneck의 bn1.weight
layer4.0.bn1.weight의 gammm one setting 완료

layer4의 0번째-bottleneck의 bn1.bias
layer4.0.bn1.bias의 beta zero setting 완료

layer4의 0번째-bottleneck의 bn2.weight
layer4.0.bn2.weight의 gammm one setting 완료

layer4의 0번째-bottleneck의 bn2.bias
layer4.0.bn2.bias의 beta zero setting 완료

layer4의 0번째-bottleneck의 bn3.weight
layer4.0.bn3.weight의 gammm zero setting 완료

layer4의 0번째-bottleneck의 bn3.bias
layer4.0.bn3.bias의 beta zero setting 완료

Epoch 0/19
----------
val Loss: 0.7906 Acc: 35.6667

데이터셋A: 300
False Recall: 10.5
False Precision: 60.0
False ACC: 35.67
False F1 score: 17.87

데이터셋B: 300
Pneumonia Recall: 86.0
Pneumonia Precision: 32.45
Pneumonia ACC: 35.67
Pneumonia F1 score: 47.12


Epoch 1/19
----------
val Loss: 0.3237 Acc: 86.3333

데이터셋A: 300
False Recall: 83.5
False Precision: 95.43
False ACC: 86.33
False F1 

'\nif epoch < 6:\n    # freeze backbone layers\n    for param in net.parameters():\n        count +=1\n        if count < 4: #freezing first 3 layers\n            param.requires_grad = False    \n        else:\n            for param in net.parameters():\n                aram.requires_grad = True\n'

# Seed 2 _  Mistake code _ layer 4 bottindex 0 

In [1]:
# Author: Sasank Chilamkurthy

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import numpy as np
from openpyxl import Workbook
from warmup_scheduler import GradualWarmupScheduler
# from pytorch_cosine_annealing_warmup_master import cosine_annearing_with_warmup
import random


plt.ion()   # interactive mode

#torch.set_deterministic(True)
random_seed = 2
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.ColorJitter(brightness=(0.8, 1.2), contrast=(0.8, 1.3)),
        transforms.RandomAffine(degrees=0, translate=(0.15,0.1)),
        transforms.RandomAffine(degrees=(-10,10)),
        transforms.Resize(280),
        transforms.CenterCrop(224),
        #transforms.Resize(size=(224,224)),
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        transforms.Normalize([0.493, 0.493, 0.493], [0.246, 0.246, 0.246])
    ]),
    'val': transforms.Compose([
        #transforms.Resize(size=(224,224)),
        transforms.Resize(280),
        transforms.CenterCrop(224),
        #transforms.Resize(size=(224,224)),
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        transforms.Normalize([0.493, 0.493, 0.493], [0.246, 0.246, 0.246])
    ]),
}

data_dir = '/home/aiffel/Hackathon_covid_19/data_2'
#data_dir = 'Crop_CXR/binary_covid'
batch_size = 16
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                             shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
val_class_names = image_datasets['val'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print("Training dataset size: " + str(dataset_sizes['train']))
print("Validation dataset size: " +str(dataset_sizes['val']))
print(val_class_names)


val_dir = '/home/aiffel/Hackathon_covid_19/data_2/val/'
false_data_dir = val_dir + 'false'
pneumonia_data_dir = val_dir + 'pneumonia'

print("######### Validation Dataset #########")
val_false_num = len(os.listdir(false_data_dir))
val_pneumonia_num = len(os.listdir(pneumonia_data_dir))
print("false size: " + str(val_false_num))
print("pneumonia size: " + str(val_pneumonia_num))

wb = Workbook()      # 워크북을 생성한다.
ws = wb.active       # 워크 시트를 얻는다.
    
ws['A1'] = 'ResNet50'
ws['B1'] = 'Val ACC'

ws['D1'] = 'False ACC'
ws['E1'] = 'False Recall'
ws['F1'] = 'False Precision'
ws['G1'] = 'False F1'

ws['I1'] = 'Penumonia ACC'
ws['J1'] = 'Penumonia Recall'
ws['K1'] = 'Penumonia Precision'
ws['L1'] = 'Penumonia F1'


def train_model(model, criterion, optimizer, scheduler, num_epochs):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    best_false_acc = 0.0
    best_false_f1 = 0.0
    best_pneumonia_acc = 0.0
    best_pneumonia_f1 = 0.0

    best_false_acc_epoch = 0
    best_false_f1_epoch = 0
    best_pneumonia_acc_epoch = 0
    best_pneumonia_f1_epoch = 0

    for epoch in range(num_epochs):
        val_false_TP = 0.0
        val_pneumonia_TP = 0.0
        val_false_FN = 0.0
        val_pneumonia_FN = 0.0
        val_false_TN = 0.0
        val_pneumonia_TN = 0.0
        val_false_FP = 0.0
        val_pneumonia_FP = 0.0

        
        
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        A = 'A'+str(epoch+2)
        ws[A] = 'Epoch' + str(epoch+1)
    
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    #outputs = torch.nn.functional.softmax(outputs, dim=1)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    # Append batch prediction results

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
                a = preds.size()
                b = a[0]
                
                if phase == 'val':
                    for i in range(b):
                        if preds[i].item() == labels.data[i].item():
                            #Normal-index:0, Pneumonia-index:1
                            if preds.data[i].item() == 0: 
                                val_false_TP += 1 #Normal 관점에서는 normal를 정확히 분류하는 것이 TP
                                val_pneumonia_TN += 1 #Pneumonia 관점에서는 normal를 정확히 분류하는 것이 TN
                            elif preds.data[i].item() == 1: 
                                val_pneumonia_TP += 1 #Pneumonia 관점에서는 Pneumonia를 정확히 분류하는 것이 TP
                                val_false_TN += 1 #Normal 관점에서는 Pneumonia를 정확히 분류하는 것이 TN

                        if preds[i].item() != labels.data[i].item():
                            if preds.data[i].item() == 0:
                                val_false_FP += 1 #Normal 관점에서 normal라고 분류했지만 실제로는 pneumonia인 경우는 FP
                                val_pneumonia_FN += 1 #Pneumonia 관점에서 normal라고 분류했지만 실제로는 pneumonia인 경우는 FN
                            elif preds.data[i].item() == 1:
                                val_pneumonia_FP += 1 #Normal 관점에서 pneumonia라고 분류했지만 실제로는 normal인 경우는 FP
                                val_false_FN += 1 #Pneumonia 관점에서 pneumonia라고 분류했지만 실제로는 normal인 경우는 FN

            if phase == 'train':
                scheduler.step()
                
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
           
            if phase == 'val':
                B = 'B'+str(epoch+2)
                ws[B] = '{:.4f}'.format(epoch_acc*100)

                print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, (epoch_acc*100)))    

            
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
#                 best_model_wts = copy.deepcopy(model.state_dict())
            

        #recall, precision -> https://en.wikipedia.org/wiki/Precision_and_recall
        #recall = TP/(TP+FN)
        #precision = TP/(TP+FP)
        #ACC = (TP+TN)/(TP+TN+FP+FN)
        false_recall = round(val_false_TP/(val_false_TP + val_false_FN),4)*100
        pneumonia_recall = round(val_pneumonia_TP/(val_pneumonia_TP + val_pneumonia_FN),4)*100
        false_precision = round(val_false_TP/(val_false_TP + val_false_FP),4)*100
        pneumonia_precision = round(val_pneumonia_TP/(val_pneumonia_TP + val_pneumonia_FP),4)*100
        false_acc = round((val_false_TP + val_false_TN)/(val_false_TP + val_false_FP +val_false_TN + val_false_FN),4)*100
        pneumonia_acc = round((val_pneumonia_TP + val_pneumonia_TN)/(val_pneumonia_TP + val_pneumonia_FP +val_pneumonia_TN + val_pneumonia_FN),4)*100
        #F1 score -> https://en.wikipedia.org/wiki/F-score
        #F1 score = 2/((1/recall)+(1/precision)) = 2((precision*recall)/(precision+recall)) = tp/(tp+((1/2)(fp+fn)))
        false_f1_score = round(2*(false_precision*false_recall)/(false_precision+false_recall),2)
        pneumonia_f1_score = round(2*(pneumonia_precision*pneumonia_recall)/(pneumonia_precision+pneumonia_recall),2)

        #total_data = batch size --> covid_total_dataset = false_total_dataset
        false_total_dataset = int(val_false_TP + val_false_FP +val_false_TN + val_false_FN)
        pneumonia_total_dataset = int(val_pneumonia_TP + val_pneumonia_FP +val_pneumonia_TN + val_pneumonia_FN)
        
      
        D = 'D'+str(epoch+2)
        E = 'E'+str(epoch+2)
        F = 'F'+str(epoch+2)
        G = 'G'+str(epoch+2)
        
        I = 'I'+str(epoch+2)
        J = 'J'+str(epoch+2)
        K = 'K'+str(epoch+2)
        L = 'L'+str(epoch+2)

        ws[D] = false_acc
        ws[E] = false_recall
        ws[F] = false_precision
        ws[G] = false_f1_score
        
        ws[I] = pneumonia_acc
        ws[J] = pneumonia_recall
        ws[K] = pneumonia_precision
        ws[L] = pneumonia_f1_score


        print()

        print('데이터셋A: ' + str(false_total_dataset))
        print('False Recall: ' + str(false_recall))        
        print('False Precision: ' + str(false_precision))
        print('False ACC: ' + str(false_acc))
        print('False F1 score: ' + str(false_f1_score))
        
        print()
        
        print('데이터셋B: ' + str(pneumonia_total_dataset))
        print('Pneumonia Recall: ' + str(pneumonia_recall))
        print('Pneumonia Precision: ' + str(pneumonia_precision))
        print('Pneumonia ACC: ' + str(pneumonia_acc))
        print('Pneumonia F1 score: ' + str(pneumonia_f1_score))
        
  
        if phase == 'val' and false_acc > best_false_acc:
            best_false_acc = false_acc
            best_false_acc_epoch = epoch

        if phase == 'val' and pneumonia_acc > best_pneumonia_acc:
            best_pneumonia_acc = pneumonia_acc
            best_pneumonia_acc_epoch = epoch
            

        if phase == 'val' and false_f1_score > best_false_f1:
            best_false_f1 = false_f1_score
            best_false_f1_epoch = epoch
            
        if phase == 'val' and pneumonia_f1_score > best_pneumonia_f1:
            best_pneumonia_f1 = pneumonia_f1_score
            best_pneumonia_f1_epoch = epoch
            best_model_wts = copy.deepcopy(model.state_dict())

        print()
        print()

    
    ws['N1'] = 'Best False ACC'
    ws['O1'] = 'epoch'
    ws['N2'] = round(best_false_acc,2)
    ws['O2'] = best_false_acc_epoch+1
        
    ws['N4'] = 'Best False F1'
    ws['O4'] = 'epoch'
    ws['N5'] = round(best_false_f1,2)
    ws['O5'] = best_false_f1_epoch+1
        
    ws['N7'] = 'Best Pneumonia ACC'
    ws['O7'] = 'epoch'
    ws['N8'] = round(best_pneumonia_acc,2)
    ws['O8'] = best_pneumonia_acc_epoch+1
        
    ws['N10'] = 'Best Pneumonia F1'
    ws['O10'] = 'epoch'
    ws['N11'] = round(best_pneumonia_f1,2)
    ws['O11'] = best_pneumonia_f1_epoch+1

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    #print('Best val Acc: {:4f}'.format(best_acc))


   

    # load best model weights
    #model.load_state_dict(best_covid_wts)
#     wb.save('output_excel/현수님세팅_Pneumonia_vs_Fasle(train5475_P 기준세팅)_Balanced(val)_seed_2_Mistake_layer_4_bottleneck_0.xlsx') # 엑셀로 저장한다. 
    torch.save(model.state_dict(), 'Final_Pneumonia_binary.pt')
    return model


model_ft = models.resnet50(pretrained=True)


###BatchNorm Gamma setting###

count =0
layer_index=4
layer_name = 'layer'+str(layer_index)

for name, layer in model_ft.named_children():
    if name == layer_name:
        bottleneck_index = 0
        bn_index = 1
        for name, param in model_ft.named_parameters():
            if name == layer_name + '.' + str(bottleneck_index) + '.' + 'bn1.weight':
                print(str(layer_name + "의 " + str(bottleneck_index)+ "번째-bottleneck의 " + 'bn1.weight'))
                torch.nn.init.ones_(param)
                print(name + '의 gammm one setting 완료')
                print()
                
            elif name == layer_name + '.' + str(bottleneck_index) + '.' + 'bn2.weight':
                print(str(layer_name + "의 " + str(bottleneck_index)+ "번째-bottleneck의 " + 'bn2.weight'))
                torch.nn.init.ones_(param)
                print(name + '의 gammm one setting 완료')
                print()

            elif name == layer_name + '.' + str(bottleneck_index) + '.' + 'bn3.weight':
                print(str(layer_name + "의 " + str(bottleneck_index)+ "번째-bottleneck의 " + 'bn3.weight'))
                torch.nn.init.ones_(param)
                print(name + '의 gammm zero setting 완료')
                print()

            elif name == layer_name + '.' + str(bottleneck_index) + '.' + 'bn' + str(bn_index) + '.bias':
                print(str(layer_name + "의 " + str(bottleneck_index)+ "번째-bottleneck의 " + 'bn' + str(bn_index) + '.bias'))
                torch.nn.init.zeros_(param)
                bn_index = bn_index + 1
                print(name + '의 beta zero setting 완료')
                print()



        bottleneck_index = bottleneck_index + 1

        layer_index = layer_index+ 1
        layer_name = 'layer'+str(layer_index)     

for param in model_ft.parameters():
    param.requires_grad = True

num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 2)




model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=3, gamma=0.1)
#exp_lr_scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max, eta_min=0, last_epoch=-1, verbose=False)
exp_lr_scheduler = GradualWarmupScheduler(optimizer_ft, multiplier=1, total_epoch=5, after_scheduler=exp_lr_scheduler)
#exp_lr_scheduler = cosine_annearing_with_warmup.CosineAnnealingWarmUpRestarts(optimizer_ft, T_0=30, T_mult=1, eta_max=0.001, T_up=20)

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=20)

# freeze backbone layers
'''
count = 0
for child_name, param in model_ft.named_children():
    for param_name, param_param in model_ft.named_parameters():
        if child_name == 'layer4': 
            param_param.requires_grad = True
        else:
            param_param.requires_grad = False
'''


#특정 conv layer 이후 초기화
'''
children_index = 4
children_name = 'layer'+str(children_index)
bottleneck_index = 2
for name, layer in model_ft.named_children():
    if name == children_name:
        conv_index = 1
        for name, param in model_ft.named_parameters():
            if name == children_name + "." + str(bottleneck_index) + '.' + 'conv' + str(conv_index) + '.weight':
                print(children_name + "의 "+ str(bottleneck_index)+ "번째-bottleneck의 " + 'conv' +str(conv_index) + '.weight')
                torch.nn.init.xavier_uniform_(param)
                print(name + '의 conv filter initilization setting 완료')
                print()
                conv_index = conv_index + 1
                if name == 'layer3.' + str(bottleneck_index) + '.' + 'conv3.weight':
                    bottleneck_index = bottleneck_index + 1
                    conv_index = 1
                elif name == 'layer4.' + str(bottleneck_index) + '.' + 'conv3.weight':
                    bottleneck_index = bottleneck_index + 1
                    conv_index = 1
        children_index = children_index + 1
        children_name = 'layer'+str(children_index)
        bottleneck_index = 0
'''        




#increase fine-tune layer per epoch
'''
if epoch < 6:
    # freeze backbone layers
    for param in net.parameters():
        count +=1
        if count < 4: #freezing first 3 layers
            param.requires_grad = False    
        else:
            for param in net.parameters():
                aram.requires_grad = True
'''

Training dataset size: 10950
Validation dataset size: 300
['false', 'pneumonia']
######### Validation Dataset #########
false size: 200
pneumonia size: 100
layer4의 0번째-bottleneck의 bn1.weight
layer4.0.bn1.weight의 gammm one setting 완료

layer4의 0번째-bottleneck의 bn1.bias
layer4.0.bn1.bias의 beta zero setting 완료

layer4의 0번째-bottleneck의 bn2.weight
layer4.0.bn2.weight의 gammm one setting 완료

layer4의 0번째-bottleneck의 bn2.bias
layer4.0.bn2.bias의 beta zero setting 완료

layer4의 0번째-bottleneck의 bn3.weight
layer4.0.bn3.weight의 gammm zero setting 완료

layer4의 0번째-bottleneck의 bn3.bias
layer4.0.bn3.bias의 beta zero setting 완료

Epoch 0/19
----------
val Loss: 0.7479 Acc: 37.3333

데이터셋A: 300
False Recall: 27.0
False Precision: 56.25
False ACC: 37.330000000000005
False F1 score: 36.49

데이터셋B: 300
Pneumonia Recall: 57.99999999999999
Pneumonia Precision: 28.43
Pneumonia ACC: 37.330000000000005
Pneumonia F1 score: 38.16


Epoch 1/19
----------
val Loss: 0.3351 Acc: 84.3333

데이터셋A: 300
False Recall: 81.0
False Pre

'\nif epoch < 6:\n    # freeze backbone layers\n    for param in net.parameters():\n        count +=1\n        if count < 4: #freezing first 3 layers\n            param.requires_grad = False    \n        else:\n            for param in net.parameters():\n                aram.requires_grad = True\n'